# Why TKET

Transpilation is the process of rewriting a given input circuit to match the topology of a specific quantum device and/or to optimize the circuit for execution on present-day noisy quantum systems.

Rewriting quantum circuits to match hardware constraints and optimizing for performance can be far from trivial. Most circuits must undergo a series of transformations that make them compatible with a given target device and optimize them to reduce the effects of noise on the resulting outcomes. The flow of logic in the rewriting toolchain need not be linear and can often have iterative sub-loops, conditional branches, and other complex behaviors. 

In this short demonstration, I want to show you the difference transpiler passes can make in circuit optimization.

We start with a sample chemistry-inspired quantum circuit obtained from the the Jordan-Wigner Mapper for computing the ground state energy of H$_2$. This 4-qubit circuit is a deep circuit. It has 150 gates, 86 layers, and 56 Xnot gates (CX).

In [21]:
from pytket.qasm import circuit_from_qasm
from pytket.circuit import OpType
circ = circuit_from_qasm('H2JordanWignerMapper.qasm')

print(f"Number of gates = {circ.n_gates}")
print(f"Circuit depth = {circ.depth()}")
print(f"Number of CX gates = {circ.n_gates_of_type(OpType.CX)}")

Number of gates = 150
Circuit depth = 83
Number of CX gates = 56


# Qiskit Preset Pass Managers

Let's optimize this circuit using the Qiskit transpiler for the noise profile of IBM' ibmq_manila's device.

First, convert the pytket circuit to a Qiskit circuit, load your IBMQ account, and get access to ibmq_manila.

In [22]:
from pytket.extensions.qiskit import qiskit_to_tk, tk_to_qiskit
cq = tk_to_qiskit(circ)

In [23]:
from qiskit import IBMQ, transpile, QuantumCircuit
#from qiskit import QuantumCircuit, QuantumRegister, IBMQ, execute

In [30]:
# load IBMQ Account data

# IBMQ.save_account(TOKEN)  # replace TOKEN with your API token string (https://quantum-computing.ibm.com/lab/docs/iql/manage/account/ibmq)
 provider = IBMQ.load_account()

In [25]:
# Get backend for experiment
provider = IBMQ.get_provider(hub='ibm-q')
manila = provider.get_backend('ibmq_manila')
# properties = manila.properties()

In [26]:
# Simulated backend based on ibmq_manila's device noise profile
from qiskit.providers.aer import QasmSimulator
sim_noisy_manila = QasmSimulator.from_backend(manila)


Now, Qiskit comes with several pre-defined pass managers, corresponding to various optimization levels achieved through different pipelines of passes. Currently, optimization_level 0 through 3 are supported; the higher the number, the more optimized it is, at the expense of more time. Choosing a good pass manager may take trial and error, depending heavily on the circuit being transpiled and the targeted backend.

optimization_level=0: just maps the circuit to the backend, with no explicit optimization (except whatever optimizations the mapper does).

optimization_level=1: maps the circuit but also does light-weight optimizations by collapsing adjacent gates.

optimization_level=2: medium-weight optimization, including a noise-adaptive layout and a gate-cancellation procedure based on gate commutation relationships.

optimization_level=3: heavy-weight optimization, which in addition to previous steps, does resynthesis of two-qubit blocks of gates in the circuit.

In [27]:
optimized_3 = transpile(cq, backend=sim_noisy_manila, seed_transpiler=11, optimization_level=3)
print('gates = ', optimized_3.count_ops())
print('depth = ', optimized_3.depth())

gates =  OrderedDict([('rz', 73), ('sx', 41), ('cx', 37), ('x', 7)])
depth =  78


We see that Qiskit's preset pass manager with optimization_level=3 reduced the circuit to 78 layers (from 83). It also reduced the number of CX gates to 37 (from 56) for ibmq_manila.

# TKET's Preset Pass Managers

Now, let's apply TKET's present pass manager `get_compiled_circuit(circuit,optimisation_level)`. As Qiskit passes, optimisation_level refers to the optimization level performed during compilation.  

Level 0 just solves the device constraints without optimising. 

Level 1 additionally performs some light optimisations. (This level is the default level)

Level 2 adds more intensive optimisations that can increase compilation time for large circuits. 

We will apply the highest preset optimization level, level 2, to the circuit for ibmq_manila.

In [28]:
from pytket.extensions.qiskit import IBMQEmulatorBackend
from qiskit import IBMQ

IBM_backend = IBMQEmulatorBackend('ibmq_manila', hub='ibm-q', group='open', project='main')

compiled_circIBM = IBM_backend.get_compiled_circuit(circ,2)   #add optimization

In [29]:
from pytket.circuit import OpType

print('Circuit depth:', compiled_circIBM.depth())
print('CX depth:     ', compiled_circIBM.depth_by_type(OpType.CX))

Circuit depth: 66
CX depth:      25


TKET's highest default optimization pass performed better than Qiskit's highest default pass. TKET reduced the initial 83 layers to 66 and the initial 56 CX gates to 25.

Here is a summary of the results in a table:

| Default Transpiler Pass | Total Layers | Number of CX gates |
| --- | --- | --- |
| None | 83 | 56 | 
| Qiskit - level 3 | 78 | 37 | 
| TKET - level 2 | 66 | 35 | 

### Now in today's NISQ area, this difference in reduction of CX gates can matter, deciding between the success and failure of implementing a circuit on a real quantum computer.